<a href="https://www.pieriandata.com"><img src="../Pierian_Data_Logo.PNG"></a>
<strong><center>Copyright by Pierian Data Inc.</center></strong> 
<strong><center>Created by Jose Marcial Portilla.</center></strong>

# Tensorboard

---
---

**NOTE: You must watch the corresponding video to understand this lecture. This notebook can't serve as a full guide. Please watch the video BEFORE posting questions to the QA forum.**

---
---

Let's explore the built-in data visualization capabilities that come with Tensorboard. We will be using the **trained** model from the classification Keras model lecture (and notebook), i.e. dataset of malign vs benign cancer tumours.

Full official tutorial available here: https://www.tensorflow.org/tensorboard/get_started

## Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../DATA/cancer_classification.csv')

In [3]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### Train Test Split

In [4]:
X = df.drop('benign_0__mal_1', axis=1).values

y = df['benign_0__mal_1'].values

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)


### Scaling Data

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [9]:
scaler.fit(X_train)

MinMaxScaler()

In [10]:
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

## Creating the Model

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

In [12]:
# You can include early-stopping at the same time, but it is not mandatory

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [13]:
# Early-stopping callback used in previous lecture (02-Keras-Classification model)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [14]:
# File path to current notebook

pwd

'C:\\Users\\shmel\\DataScience\\bootcamp_course\\Py_DS_ML_Bootcamp-master\\22-TensorFlow\\ANNs'

## Creating the Tensorboard Callback

TensorBoard is a visualization tool provided with TensorFlow.

This callback logs events for TensorBoard, including:
* Metrics summary plots
* Training graph visualization
* Activation histograms
* Sampled profiling

If you have installed TensorFlow with pip, you should be able
to launch TensorBoard from the command line:

```sh
tensorboard --logdir=path_to_your_logs
```

You can find more information about TensorBoard
[here](https://www.tensorflow.org/tensorboard/).

    Arguments:
        log_dir: the path of the directory where to save the log files to be
          parsed by TensorBoard.
        histogram_freq: frequency (in epochs) at which to compute activation and
          weight histograms for the layers of the model. If set to 0, histograms
          won't be computed. Validation data (or split) must be specified for
          histogram visualizations.
        write_graph: whether to visualize the graph in TensorBoard. The log file
          can become quite large when write_graph is set to True.
        write_images: whether to write model weights to visualize as image in
          TensorBoard.
        update_freq: `'batch'` or `'epoch'` or integer. When using `'batch'`,
          writes the losses and metrics to TensorBoard after each batch. The same
          applies for `'epoch'`. If using an integer, let's say `1000`, the
          callback will write the metrics and losses to TensorBoard every 1000
          samples. Note that writing too frequently to TensorBoard can slow down
          your training.
        profile_batch: Profile the batch to sample compute characteristics. By
          default, it will profile the second batch. Set profile_batch=0 to
          disable profiling. Must run in TensorFlow eager mode.
        embeddings_freq: frequency (in epochs) at which embedding layers will
          be visualized. If set to 0, embeddings won't be visualized.
       

In [15]:
from datetime import datetime

In [16]:
# How to use datetime to return the current date and time

datetime.now().strftime("%Y-%m-%d--%H%M")

'2023-05-05--1042'

In [17]:
# WINDOWS: Use "logs\\fit" (double backslash to match syntax from file path above)

log_directory = 'logs\\fit'

# OPTIONAL: ADD A TIMESTAMP TO CREATE UNIQUE FOLDER EACH TIME YOU TRAIN MODEL, in case you change something like no of neurons
# EACH NEW FOLDER IS NAMED WITH UNIQUE DATE/TIME WHEN MODEL WAS TRAINED
#timestamp = datetime.now().strftime("%Y-%m-%d--%H%M")
#log_directory = log_directory + '\\' + timestamp

board = TensorBoard(log_dir=log_directory, 
                    histogram_freq=1, 
                    write_graph=True, 
                    write_images=True, 
                    update_freq='epoch', 
                    profile_batch=2, 
                    embeddings_freq=1)

Now create the model layers (with drop-out layers used in previous classification model):

In [18]:
model = Sequential()

model.add(Dense(units=30, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=15, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

## Train the Model

In [19]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600, 
          validation_data=(X_test, y_test), 
          verbose=1, 
          callbacks=[early_stop, board])

Epoch 1/600
14/14 [==============================] - 1s 41ms/step - loss: 0.6808 - val_loss: 0.6702
Epoch 2/600
14/14 [==============================] - 0s 12ms/step - loss: 0.6732 - val_loss: 0.6573
Epoch 3/600
14/14 [==============================] - 0s 10ms/step - loss: 0.6419 - val_loss: 0.6395
Epoch 4/600
14/14 [==============================] - 0s 12ms/step - loss: 0.6426 - val_loss: 0.6162
Epoch 5/600
14/14 [==============================] - 0s 11ms/step - loss: 0.6238 - val_loss: 0.5897
Epoch 6/600
14/14 [==============================] - 0s 10ms/step - loss: 0.6086 - val_loss: 0.5657
Epoch 7/600
14/14 [==============================] - 0s 13ms/step - loss: 0.5950 - val_loss: 0.5414
Epoch 8/600
14/14 [==============================] - 0s 12ms/step - loss: 0.5795 - val_loss: 0.5168
Epoch 9/600
14/14 [==============================] - 0s 10ms/step - loss: 0.5439 - val_loss: 0.4881
Epoch 10/600
14/14 [==============================] - 0s 10ms/step - loss: 0.5201 - val_loss: 0.4554

14/14 [==============================] - 0s 10ms/step - loss: 0.1056 - val_loss: 0.0977
Epoch 83/600
14/14 [==============================] - 0s 10ms/step - loss: 0.1278 - val_loss: 0.0937
Epoch 84/600
14/14 [==============================] - 0s 10ms/step - loss: 0.1177 - val_loss: 0.0893
Epoch 85/600
14/14 [==============================] - 0s 10ms/step - loss: 0.1083 - val_loss: 0.1053
Epoch 86/600
14/14 [==============================] - 0s 11ms/step - loss: 0.1074 - val_loss: 0.1054
Epoch 87/600
14/14 [==============================] - 0s 9ms/step - loss: 0.1145 - val_loss: 0.0932
Epoch 88/600
14/14 [==============================] - 0s 11ms/step - loss: 0.1236 - val_loss: 0.0910
Epoch 89/600
14/14 [==============================] - 0s 10ms/step - loss: 0.1180 - val_loss: 0.0909
Epoch 90/600
14/14 [==============================] - 0s 10ms/step - loss: 0.1169 - val_loss: 0.0960
Epoch 91/600
14/14 [==============================] - 0s 11ms/step - loss: 0.1174 - val_loss: 0.1289
Epoc

# Running Tensorboard


## Running through the Command Line

**Watch video to see how to run Tensorboard through Anaconda Prompt command line.**

Tensorboard will run locally in your browser at [http://localhost:6006/](http://localhost:6006/) Click on link to open empty browser page. TensorBoard dashboard will appear when command line is run.


In [20]:
# To access the correct file path to log files

print(log_directory)

logs\fit


In [21]:
# To access the correct file path to current working folder (where log files are stored)

pwd

'C:\\Users\\shmel\\DataScience\\bootcamp_course\\Py_DS_ML_Bootcamp-master\\22-TensorFlow\\ANNs'

### 1. Use `cd` in your command line to change directory to the file path reported back by pwd (your current location where logs folder exists).

### 2. Then run this command at your terminal

`tensorboard --logdir logs\fit`

**Note:** To close the TensorBoard after using, press CTRL+C at your terminal. Once you have quit the call, you will need to re-train your model again to see it.